In [2]:
# === 1. Import Required Libraries ===
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# Only accuracy_score is needed from sklearn.metrics
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Import Keras components from TensorFlow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Suppress unnecessary warnings and TensorFlow messages
warnings.filterwarnings('ignore', category=UserWarning)
tf.get_logger().setLevel('ERROR')

# === 2. Load Dataset ===
try:
    combined_dataset = pd.read_csv("/Users/administrator/Desktop/All desktop-data/Prof-Irshad Papers/2025-Papers/10K-Paper/Crop-IoT-Paper/IoT-Journal-Submission/Manuscript-R2/Code-For-Github/Combined_Data.csv")
except FileNotFoundError:
    print("Error: CSV file not found. Using a dummy dataframe for demonstration.")
    data = np.random.rand(5000, 9)
    columns = ['N', 'P', 'K', 'Temp', 'H', 'pH', 'R', 'WL', 'CH']
    combined_dataset = pd.DataFrame(data, columns=columns)
    combined_dataset['CH'] = np.random.randint(0, 2, 5000)

# === 3. Prepare, Split, Scale, and Balance Data ===
features = ['N', 'P', 'K', 'Temp', 'H', 'pH', 'R', 'WL']
n_features = len(features)
X = combined_dataset[features].values
y = combined_dataset['CH'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_scaled, y_train)

# === 4. Reshape Data for 1D CNN ===
X_train_reshaped = X_train_resampled.reshape(X_train_resampled.shape[0], n_features, 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], n_features, 1)


# === 5. Define a function to build the CNN model ===
def build_cnn_model():
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_features, 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# === 6. Run Simulation for CNN Model (Tracking Accuracy Only) ===
EPOCHS = 163
acc_list = [] # Only need a list for accuracy

n_samples = len(X_train_reshaped)
print(f"\nStarting 1D CNN simulation for {EPOCHS} epochs...")
print("-" * 50)

for epoch in range(1, EPOCHS + 1):
    subset_size = int(n_samples * (epoch / EPOCHS))
    subset_size = max(10, subset_size)
    
    X_subset = X_train_reshaped[:subset_size]
    y_subset = y_train_resampled[:subset_size]

    if len(np.unique(y_subset)) < 2:
        print(f"Epoch {epoch:03d}/{EPOCHS}: Skipped (not enough class diversity)")
        acc_list.append(acc_list[-1] if acc_list else 0.5)
        continue

    # --- Build and Train a new CNN Model on the current subset ---
    cnn_model = build_cnn_model()
    cnn_model.fit(X_subset, y_subset, epochs=10, batch_size=32, verbose=0)
    
    # --- Evaluate on the FULL, UNSEEN test set ---
    y_pred = (cnn_model.predict(X_test_reshaped, verbose=0) >= 0.5).astype(int)
    
    # --- Calculate and Store Accuracy ---
    acc = accuracy_score(y_test, y_pred)
    acc_list.append(acc)
    
    # --- Print Only Accuracy ---
    print(f"Epoch {epoch:03d}/{EPOCHS} -> Accuracy: {acc*100:.2f}%")


# === 7. Store History and Print Final Result ===
metrics_df = pd.DataFrame({
    'Epoch': range(1, EPOCHS + 1),
    'Accuracy': acc_list
})

final_accuracy = metrics_df['Accuracy'].iloc[-1]

print("\n" + "="*50)
print("Simulation Complete.")
print(f"Final Accuracy at Epoch {EPOCHS}: {final_accuracy*100:.2f}%")
print("="*50)


Starting 1D CNN simulation for 163 epochs...
--------------------------------------------------
Epoch 001/163: Skipped (not enough class diversity)
Epoch 002/163: Skipped (not enough class diversity)
Epoch 003/163: Skipped (not enough class diversity)
Epoch 004/163: Skipped (not enough class diversity)
Epoch 005/163: Skipped (not enough class diversity)
Epoch 006/163: Skipped (not enough class diversity)
Epoch 007/163: Skipped (not enough class diversity)
Epoch 008/163: Skipped (not enough class diversity)
Epoch 009/163: Skipped (not enough class diversity)
Epoch 010/163: Skipped (not enough class diversity)
Epoch 011/163: Skipped (not enough class diversity)
Epoch 012/163: Skipped (not enough class diversity)
Epoch 013/163: Skipped (not enough class diversity)
Epoch 014/163: Skipped (not enough class diversity)
Epoch 015/163 -> Accuracy: 99.55%
Epoch 016/163 -> Accuracy: 99.55%
Epoch 017/163 -> Accuracy: 99.55%
Epoch 018/163 -> Accuracy: 99.55%
Epoch 019/163 -> Accuracy: 99.55%
Epoch